In [59]:
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import re
import csv

In [60]:
# import the story list
# resave the csv file, because the orignial csv may contain the error chars
df = pd.read_csv("story_listUS_2.csv" ,encoding="ISO-8859-1" )
df.head()

,stories_id,publish_date,title,url,language,ap_syndicated,themes,media_id,media_name,media_url
0,24842462,2010/10/1 3:15,Ad watch | Robin Carnahan and Roy Blunt,http://www.kansascity.com/2010/09/30/2264637/a...,NaN,False,NaN,30,Kansas City Star,http://www.kansascity.com/
1,24846848,2010/10/1 3:15,Younger Frederick enters the BCS fray,http://www.kansascity.com/2010/09/30/2265196/y...,NaN,False,NaN,30,Kansas City Star,http://www.kansascity.com/
2,24852751,2010/10/1 3:15,"NASCAR's Stewart can be difficult, but ther...",http://www.kansascity.com/2010/09/30/2265128/n...,NaN,False,NaN,30,Kansas City Star,http://www.kansascity.com/
3,24854379,2010/10/1 3:15,"When Washington fails, average citizens ste...",http://www.kansascity.com/2010/09/30/2264678/w...,NaN,False,NaN,30,Kansas City Star,http://www.kansascity.com/
4,24855301,2010/10/1 3:15,"Movie Mom | &lsquo;The Social Network,' &ls...",http://www.kansascity.com/2010/09/30/2262171/m...,NaN,False,NaN,30,Kansas City Star,http://www.kansascity.com/


In [61]:
# import the tags
tags = pd.read_csv("Tags2.csv")
tags.tail()

,Source,Source_ID,URL_Root,Key_for_article,Notes
164,Kansas City Star,30,http://www.kansascity.com/,NaN,Unreachable
165,Baltimore Sun,34,http://www.baltimoresun.com/,"(""div"", class_=""crd clln--it"")",NaN
166,San Jose Mercury News,35,http://www.mercurynews.com/,NaN,Unreachable
167,Milwaukee Journal Sentinel,36,http://www.jsonline.com/,(p),NaN
168,Orlando Sentinal,38,feeds.orlandosentinal.com,"every (""p"", class = "" "") within (""div"", class ...",NaN


In [62]:
# convert the Source ID into integer


tags['Source_ID'] = tags['Source_ID'].astype(int)
tags['Source_ID']

0       1
1       1
2       1
3       1
4       1
       ..
164    30
165    34
166    35
167    36
168    38
Name: Source_ID, Length: 169, dtype: int32

In [63]:
# convert the media_id into integer
df['media_id'] = df['media_id'].astype(int)
df['media_id']

0        30
1        30
2        30
3        30
4        30
         ..
87223    41
87224    30
87225    30
87226    36
87227    30
Name: media_id, Length: 87228, dtype: int32

In [64]:
# identify and extract out the row of the story
for n in range(len(df)):
    eg = df.iloc[n]
    eg

In [65]:
# searching for the tags 
temp = tags[tags["Source_ID"] == eg["media_id"]]
temp

,Source,Source_ID,URL_Root,Key_for_article,Notes
164,Kansas City Star,30,http://www.kansascity.com/,NaN,Unreachable


In [66]:
full = ''
# to perform scraping loop   
for i in range(len(temp)):
    # when URL Root is contained in url, then perform the func below to extract the content of webpage
    if temp["URL Root"].iloc[i] in eg["url"]:
        key = tags["Key for article"].iloc[i]
        try:
            page = requests.get(eg["url"])
            soup = BeautifulSoup(page.content, "html.parser")
            if 'and' in key:
                key = key.split(' and ')
                contents0 = soup.find_all(key[0].strip('(" ")'))
                for content in contents0:
                    full += content.get_text()
                    contents1 = soup.find_all(key[1].strip('()'))
                for content in contents1:
                    full += content.get_text()
            else:
                contents = contents = soup.find_all(key.strip('(" ")'))
                for content in contents:
                    full += content.get_text()
            if "The page you were looking for couldn't be found" in full:
                    full = 'broken'
        except:
            full ="broken"
        break

KeyError: 'URL Root'

In [ ]:
full

In [ ]:
title = '"'+df['title'][n]+'"'
title

In [ ]:
#save the title and text to a file
save_file = open(r'result.txt', 'w')
save_file.write(title)
save_file.write(full)
save_file.close()